In [1]:
#    ____       _       _
#   |  _ \  ___| |_ __ | |__   __ _  ___
#   | | | |/ _ \ | '_ \| '_ \ / _` |/ _ \
#   | |_| |  __/ | |_) | | | | (_| |  __/
#   |____/ \___|_| .__/|_| |_|\__,_|\___|
#                |_|

In [2]:
import requests
import json
from datetime import datetime as dt
import pandas as pd

## Azure AD Authentication

https://docs.microsoft.com/nl-nl/azure/active-directory/develop/howto-create-service-principal-portal

1. create Azure AD application to 'App registrations' <br>
    for example: 'azure-lab4-arm-api' <br>
    redirect: http://localhost <br>
    create 'Client secret' and save to variable azure_secret <br>
<br>    
2. assign application to a RBAC role 'Owner'<br>
   via Cost Management + Billing, select Subscription 'Free Trial'<br>
   Access Control (IAM)<br>
   Role assignments<br>
   Search by application name 'azure-lab4-arm-api'<br>
   Add role assigment to 'Owner'
   

### Azure ID's

In [218]:
# azure-lab4-arm-api

# subscription ID can be found on a resource, overview page
azure_subscription_id = 'a8682560-0e6b-4834-accf-21ff95b29125'

# app information can be found:
# Azure Active Directory
# app Registrations
# overview
azure_tenant = '3089f1b6-d716-42a9-9dd5-a580cf7b4f12'
azure_client_id = '0407876f-fcd1-4260-bfa4-4180a54beced'

# client secret only visible during creation
# copy & pase here
azure_secret = 'y14SA6[CI5SN/caV:::K[SeH+h/09e[g'

### Request Access Token

In [219]:
# authorize with azure

#
# tutorial
# http://exponentialcoding.com/accessing-azure-rest-api-from-python/
#
# remark: access_token expires in 1 hour
#

url = "https://login.windows.net/%s/oauth2/token" % azure_tenant

#data = "resource=https%3A%2F%2Fmanagement.core.windows.net%2F&client_id=" + azure_client_id + "&grant_type=client_credentials&client_secret=" + azure_secret
parameters = {
    'resource' : 'https://management.core.windows.net',
    'client_id' : azure_client_id,
    'grant_type' : 'client_credentials',
    'client_secret': azure_secret
}

headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(url, data=parameters, headers=headers)
jsdata = response.json()

print (json.dumps(jsdata, indent=4))
access_token = jsdata['access_token']

{
    "token_type": "Bearer",
    "expires_in": "3600",
    "ext_expires_in": "3600",
    "expires_on": "1565818578",
    "not_before": "1565814678",
    "resource": "https://management.core.windows.net",
    "access_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCIsImtpZCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuY29yZS53aW5kb3dzLm5ldCIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzMwODlmMWI2LWQ3MTYtNDJhOS05ZGQ1LWE1ODBjZjdiNGYxMi8iLCJpYXQiOjE1NjU4MTQ2NzgsIm5iZiI6MTU2NTgxNDY3OCwiZXhwIjoxNTY1ODE4NTc4LCJhaW8iOiI0MkZnWUxCcloyLzJZYS8xZFlyNjhrUkFhcWM3QUE9PSIsImFwcGlkIjoiMDQwNzg3NmYtZmNkMS00MjYwLWJmYTQtNDE4MGE1NGJlY2VkIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMzA4OWYxYjYtZDcxNi00MmE5LTlkZDUtYTU4MGNmN2I0ZjEyLyIsIm9pZCI6IjYwNDU2YzQzLTI4NjktNDY5NS05YTM5LTcwM2UxZjM1NzliMSIsInN1YiI6IjYwNDU2YzQzLTI4NjktNDY5NS05YTM5LTcwM2UxZjM1NzliMSIsInRpZCI6IjMwODlmMWI2LWQ3MTYtNDJhOS05ZGQ1LWE1ODBjZjdiNGYxMiIsInV0aSI6I

## Azure Resource Explorer
https://resources.azure.com

copy & paste the URL to variable 'url'<br>
your Azure subscription_id will change after the Free Trial period<br>
copy & past the new URL and save to variable 'url' below

In [220]:
headers = {'Authorization': 'Bearer ' + access_token, 
           'Content-Type': 'application/json' }

In [221]:
# subscriptions

url = 'https://management.azure.com/subscriptions?api-version=2014-04-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))


200
[
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125",
        "subscriptionId": "a8682560-0e6b-4834-accf-21ff95b29125",
        "displayName": "Free Trial",
        "state": "Enabled",
        "subscriptionPolicies": {
            "locationPlacementId": "Public_2014-09-01",
            "quotaId": "FreeTrial_2014-09-01",
            "spendingLimit": "On"
        }
    }
]


In [222]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))

200
[
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope",
        "name": "cloud-shell-storage-westeurope",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
        "name": "delphae-freetrail",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/NetworkWatcherRG",
        "name": "NetworkWatcherRG",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    }
]


In [223]:
# delphae-freetrail

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail?api-version=2014-04-01"
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata, indent=4))

200
{
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
    "name": "delphae-freetrail",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded"
    }
}


In [225]:
# virtual machines

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Compute/virtualMachines?api-version=2018-10-01'
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))

for vm in jsdata['value']:
    print ('{:20s}{:15s}{:15s}{:10s}'.format(vm['name'], 
                           vm['location'], 
                           vm['properties']['storageProfile']['osDisk']['osType'], 
                           vm['properties']['storageProfile']['imageReference']['sku']))

200
{
    "name": "azure-ie-ubuntu-03",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Compute/virtualMachines/azure-ie-ubuntu-03",
    "type": "Microsoft.Compute/virtualMachines",
    "location": "northeurope",
    "properties": {
        "vmId": "9d4b2a6c-1548-4001-9195-0aa44bb70b4f",
        "hardwareProfile": {
            "vmSize": "Standard_B1s"
        },
        "storageProfile": {
            "imageReference": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "18.04-LTS",
                "version": "latest"
            },
            "osDisk": {
                "osType": "Linux",
                "name": "azure-ie-ubuntu-03_OsDisk_1_cecbf9731d22446296938226597069a9",
                "createOption": "FromImage",
                "caching": "ReadWrite",
                "managedDisk": {
                    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff9

In [50]:
# connections

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
#print (json.dumps(jsdata['value'][0], indent=4))
print (json.dumps(jsdata['value'][0]['properties'], indent=4))

200
{
    "provisioningState": "Succeeded",
    "resourceGuid": "28241ef5-734f-42d3-bf4b-8b31c624672d",
    "virtualNetworkGateway1": {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/virtualNetworkGateways/lab2-vnet-gw"
    },
    "localNetworkGateway2": {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan"
    },
    "connectionType": "IPsec",
    "connectionProtocol": "IKEv1",
    "routingWeight": 0,
    "enableBgp": false,
    "usePolicyBasedTrafficSelectors": false,
    "ipsecPolicies": [],
    "ingressBytesTransferred": 0,
    "egressBytesTransferred": 0,
    "expressRouteGatewayBypass": false
}


In [66]:
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()

200


In [67]:
print (json.dumps(jsdata, indent=4))

{
    "name": "lab2-meraki-vpn",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/connections/lab2-meraki-vpn",
    "etag": "W/\"7c71f338-b74a-48fa-b50c-ab9afafa579e\"",
    "type": "Microsoft.Network/connections",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded",
        "resourceGuid": "28241ef5-734f-42d3-bf4b-8b31c624672d",
        "virtualNetworkGateway1": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/virtualNetworkGateways/lab2-vnet-gw"
        },
        "localNetworkGateway2": {
            "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan"
        },
        "connectionType": "IPsec",
        "connectionProtocol": "IKEv1",
        "routingWeight": 0,
        "sharedKey"

In [73]:
properties = jsdata['properties']
print (properties['connectionStatus'])
print (properties['ingressBytesTransferred'] / 1024.0 / 1024.0)
print (properties['egressBytesTransferred'] / 1024.0 / 1024.0)

Connected
0.6651077270507812
5.507591247558594


In [74]:
# localNetworkGateways
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways?api-version=2018-07-01'

response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'][0], indent=4))


200
{
    "name": "lab2-meraki-sdwan",
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/localNetworkGateways/lab2-meraki-sdwan",
    "etag": "W/\"191f5411-3325-4403-bd60-670914c7bd61\"",
    "type": "Microsoft.Network/localNetworkGateways",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded",
        "resourceGuid": "3e7c0b0a-6e51-417a-8b47-bbe8b8dd8d27",
        "localNetworkAddressSpace": {
            "addressPrefixes": [
                "192.168.64.0/24"
            ]
        },
        "gatewayIpAddress": "84.31.151.136"
    }
}


In [78]:
localgw = jsdata['value'][0]
print ('{:20}{}'.format(localgw['name'], localgw['properties']['localNetworkAddressSpace']))
print ('{:20}{}'.format(localgw['name'], localgw['properties']['gatewayIpAddress']))

lab2-meraki-sdwan   {'addressPrefixes': ['192.168.64.0/24']}
lab2-meraki-sdwan   84.31.151.136


In [27]:
# resources

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resources?api-version=2017-05-10'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

df = pd.DataFrame(data)

200


In [28]:
df[['name','type','location']][df.type=='Microsoft.Compute/virtualMachines']

,name,type,location
6,azure-fr-ubuntu-07,Microsoft.Compute/virtualMachines,francecentral
7,azure-ie-ubuntu-03,Microsoft.Compute/virtualMachines,northeurope
8,azure-nl-ubuntu-01,Microsoft.Compute/virtualMachines,westeurope
9,azure-uk-ubuntu-04,Microsoft.Compute/virtualMachines,ukwest


In [29]:
df[['name','type','location']][df.type=='Microsoft.Network/localNetworkGateways']

,name,type,location
15,azure-asus-rtr,Microsoft.Network/localNetworkGateways,ukwest
16,lab2-meraki-sdwan,Microsoft.Network/localNetworkGateways,westeurope


In [30]:
df[['name','type','location']][df.type=='Microsoft.Network/virtualNetworkGateways']

,name,type,location
31,lab2-vnet-gw,Microsoft.Network/virtualNetworkGateways,westeurope


In [32]:
df[['name','type','location']][df.type=='Microsoft.Network/connections']

,name,type,location
14,lab2-meraki-vpn,Microsoft.Network/connections,westeurope


In [20]:
df[['name','type','location']]

,name,type,location
0,csba86825600e6bx4834xacc,Microsoft.Storage/storageAccounts,westeurope
1,azure-fr-ubuntu-07_OsDisk_1_046af4b0b18543bcbf...,Microsoft.Compute/disks,francecentral
2,azure-ie-ubuntu-03_OsDisk_1_cecbf9731d22446296...,Microsoft.Compute/disks,northeurope
3,azure-nl-ubuntu-01_disk1_d32585eddccc4135b57d4...,Microsoft.Compute/disks,westeurope
4,azure-uk-ubuntu-02_OsDisk_1_649bf025e42841feaf...,Microsoft.Compute/disks,northeurope
5,azure-uk-ubuntu-04_OsDisk_1_48bda43803444f36b4...,Microsoft.Compute/disks,ukwest
6,azure-fr-ubuntu-07,Microsoft.Compute/virtualMachines,francecentral
7,azure-ie-ubuntu-03,Microsoft.Compute/virtualMachines,northeurope
8,azure-nl-ubuntu-01,Microsoft.Compute/virtualMachines,westeurope
9,azure-uk-ubuntu-04,Microsoft.Compute/virtualMachines,ukwest


In [53]:
# public IP addresses

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail/providers/Microsoft.Network/publicIPAddresses?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

df = pd.DataFrame(data)

200


In [65]:
for item in data:
    try:
        properties = item['properties']
        print ('{:25s}{:15s}{}'.format(item['name'], item['location'], properties['ipAddress']))
    except:
        pass

azure-nl-ubuntu-01-ip    westeurope     52.166.37.193
lab2-vnet-ip             westeurope     52.178.29.168


In [208]:
# virtual networks

url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/virtualNetworks?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']
#print (json.dumps(data[0], indent=4))


for virtnet in data:
    properties = virtnet['properties']
    print ('virtualnet: {:20}{}'.format(virtnet['name'], properties['addressSpace']['addressPrefixes'][0]))
    subnets = properties['subnets']
    for subnet in subnets:
        print ('subnet: {:24}{}'.format(subnet['name'], subnet['properties']['addressPrefix']))
    print ()

200
virtualnet: azure-ie-vnet-01    10.1.0.0/16
subnet: azure-ie-subnet-01      10.1.0.0/24
subnet: GatewaySubnet           10.1.1.0/24

virtualnet: labsource-vnet      10.3.1.0/24
subnet: labsubnet               10.3.1.0/25
subnet: GatewaySubnet           10.3.1.128/28

virtualnet: azure-uk-vnet-04    10.8.0.0/16
subnet: azure-uk-subnet-06      10.8.0.0/24
subnet: GatewaySubnet           10.8.1.0/24

virtualnet: azure-fr-vnet-08    10.5.0.0/16
subnet: azure-fr-subnet-07      10.5.0.0/24



### Create a Resoure Group

In [98]:
import numpy as np

In [148]:
resource_group_name = 'azure-lab4-apitest%03d' % np.random.randint(100)

url = "https://management.azure.com/subscriptions/" + azure_subscription_id + "/resourcegroups/" + resource_group_name + "?api-version=2017-05-10"
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
body = { 'name': resource_group_name, 'location': 'westeurope' }
response = requests.put(url, data=json.dumps(body), headers=headers)
print (response.status_code)


print (json.dumps(response.json(), indent=4))

201
{
    "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/azure-lab4-apitest077",
    "name": "azure-lab4-apitest077",
    "location": "westeurope",
    "properties": {
        "provisioningState": "Succeeded"
    }
}


In [171]:
# resource groups

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
if response.status_code!=200:
    print (response.text)
jsdata = response.json()
#print (json.dumps(jsdata['value'], indent=4))
pd.DataFrame(jsdata['value'])[['name','location']]

200


,name,location
0,cloud-shell-storage-westeurope,westeurope
1,delphae-freetrail,westeurope
2,NetworkWatcherRG,westeurope


### Delete a Resoure Group

In [172]:
resource_group_name = 'azure-lab4-apitest933' 
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/' + resource_group_name + '?api-version=2014-04-01'
headers = { 'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/json' }
#body = { 'name': resource_group_name, 'location': 'westeurope' }
body = { 'name': resource_group_name}
response = requests.delete(url, data=json.dumps(body), headers=headers)
print (response.status_code)
print (response.text)

404
{"error":{"code":"ResourceGroupNotFound","message":"Resource group 'azure-lab4-apitest933' could not be found."}}


In [173]:
# ProvisioningState: Deleting

url = "https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups?api-version=2014-04-01"
#print (url)
response = requests.get(url, headers=headers, timeout=5)
print (response.status_code)
jsdata = response.json()
print (json.dumps(jsdata['value'], indent=4))
#pd.DataFrame(jsdata['value'])[['name','location']]

200
[
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/cloud-shell-storage-westeurope",
        "name": "cloud-shell-storage-westeurope",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/delphae-freetrail",
        "name": "delphae-freetrail",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    },
    {
        "id": "/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/resourceGroups/NetworkWatcherRG",
        "name": "NetworkWatcherRG",
        "location": "westeurope",
        "properties": {
            "provisioningState": "Succeeded"
        }
    }
]


### Express Route Service Providers (ERSP)

In [209]:
url = 'https://management.azure.com/subscriptions/a8682560-0e6b-4834-accf-21ff95b29125/providers/Microsoft.Network/expressRouteServiceProviders?api-version=2018-07-01'
response = requests.get(url, headers=headers)
print (response.status_code)
jsdata = response.json()
data = jsdata['value']

200


In [210]:
for ersp in data:
    if ersp['name']=='Equinix':
        print (json.dumps(ersp, indent=4))

{
    "name": "Equinix",
    "id": "/subscriptions//resourceGroups//providers/Microsoft.Network/expressRouteServiceProviders/",
    "type": "Microsoft.Network/expressRouteServiceProviders",
    "properties": {
        "provisioningState": "Succeeded",
        "peeringLocations": [
            "Amsterdam",
            "Atlanta",
            "Chicago",
            "Dallas",
            "Dublin",
            "Hong Kong",
            "London",
            "London2",
            "Los Angeles",
            "Melbourne",
            "Miami",
            "New York",
            "Osaka",
            "Paris",
            "Sao Paulo",
            "Seattle",
            "Silicon Valley",
            "Singapore",
            "Sydney",
            "Tokyo",
            "Toronto",
            "Washington DC"
        ],
        "bandwidthsOffered": [
            {
                "offerName": "50Mbps",
                "valueInMbps": 50
            },
            {
                "offerName": "100Mbps",

In [212]:
for ersp in data:
    if 'Amsterdam' in ersp['properties']['peeringLocations']:
        #print (json.dumps(ersp, indent=4))
        print (ersp['name'])

Aryaka Networks
AT&T Netbond
British Telecom
Colt Ethernet
Colt IPVPN
Equinix
euNetworks
GEANT
InterCloud
InterCloud for Azure
Interxion
IX Reach
KPN
Level 3 Communications - Exchange
Level 3 Communications - IPVPN
Megaport
NTT Communications
Orange
Tata Communications
Telefonica
Telenor
Telia Carrier
Verizon
Zayo
